<a href="https://colab.research.google.com/github/mkbahk/AmazonBraket/blob/main/VQE_Qiskit_nature_AerSim_mkbahk_20250717.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install qiskit
!pip install qiskit-nature
!pip install qiskit-aer
!pip install pyscf
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 22.9 MB/s eta 0:00:00


In [12]:
!pip install qiskit-algorithms

In [20]:
### 물 분자(H₂O) VQE 계산 예제 코드

# 1. 필요한 라이브러리 임포트
import numpy as np
import matplotlib.pyplot as plt

# Qiskit 관련 임포트
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_algorithms.optimizers import SPSA, COBYLA
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper


from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit_nature.second_q.drivers.pyscfd.pyscfdriver import PySCFDriver


In [22]:
# 2. 물 분자 구조 정의
# PySCFDriver를 사용하여 물 분자의 구조를 정의합니다.
# 분자 구조는 '원자 기호 x좌표 y좌표 z좌표' 형식의 문자열로 지정합니다.
# 단위는 옹스트롬(Angstrom)입니다.
h2o_geometry = "O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0"

In [23]:
# PySCFDriver 인스턴스 생성
# basis는 분자 오비탈을 표현하기 위한 기저 함수 집합입니다. 'sto-3g'는 계산량이 적어 예제에 적합합니다.
driver = PySCFDriver(atom=h2o_geometry, basis="sto-3g")
problem = driver.run()

In [24]:
# 3. 페르미온 해밀토니안을 큐비트 해밀토니안으로 변환
# 분자의 전자 구조 문제(페르미온)를 양자 컴퓨터가 다룰 수 있는 큐비트 문제로 변환합니다.
# ParityMapper는 JordanWignerMapper보다 필요한 큐비트 수를 줄여줄 수 있습니다.
mapper = ParityMapper(num_particles=problem.num_particles)
qubit_hamiltonian = mapper.map(problem.hamiltonian.second_q_op())

In [27]:
# 4. Ansatz (시험 파동 함수) 및 초기 상태 준비
# UCCSD Ansatz는 양자 화학에서 널리 사용되는 화학적으로 영감을 받은 Ansatz입니다.
# 초기 상태는 Hartree-Fock 상태로 설정합니다.
num_spatial_orbitals = problem.num_spatial_orbitals
num_particles = problem.num_particles

initial_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
ansatz = UCCSD(num_spatial_orbitals, num_particles, mapper, initial_state=initial_state) # Using alternative initialization

In [28]:
# 5. 백엔드(시뮬레이터) 설정
# Qiskit Aer의 Estimator를 사용하여 해밀토니안의 기댓값을 효율적으로 계산합니다.
# shot-noise 효과를 제거하기 위해 shots=None으로 설정합니다 (이상적인 시뮬레이션).
aer_estimator = AerEstimator(run_options={"shots": None, "seed": 170}, transpile_options={"seed_transpiler": 170})

In [ ]:
# 6. VQE 알고리즘 설정 및 실행
# 고전적인 최적화기(Optimizer)를 선택합니다. SPSA는 노이즈에 강한 최적화기입니다.
optimizer = SPSA(maxiter=100)

# 반복마다 비용(에너지)을 저장하기 위한 콜백 함수
cost_history = []
def store_intermediate_result(eval_count, parameters, mean, std):
    cost_history.append(mean)
    print(f"Iters. done: {eval_count} [Current cost: {mean}]")

# VQE 알고리즘 인스턴스 생성
vqe_solver = VQE(
    estimator=aer_estimator,
    ansatz=ansatz,
    optimizer=optimizer,
    callback=store_intermediate_result
)

# VQE 계산 실행
vqe_result = vqe_solver.compute_minimum_eigenvalue(qubit_hamiltonian)



/usr/local/lib/python3.11/dist-packages/qiskit_aer/primitives/estimator.py:218: RuntimeWarning: If `shots` is None and `approximation` is False, the number of shots is automatically set to backend options' shots=1024.
  warn(


In [ ]:
# 7. 결과 출력 및 시각화
print("\n--- VQE 계산 결과 ---")
print(f"최종 계산된 바닥 상태 에너지: {vqe_result.eigenvalue.real:.12f}")

# 정확한 바닥 상태 에너지 값 계산 (고전적인 방법)
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
solver = GroundStateEigensolver(mapper)
exact_result = solver.solve(problem)
print(f"정확한 바닥 상태 에너지 (고전 계산): {exact_result.total_energies[0]:.12f}")
print("--------------------")

# 최적화 과정 시각화
plt.figure(figsize=(10, 6))
plt.plot(range(len(cost_history)), cost_history, marker='o')
plt.axhline(y=exact_result.total_energies[0], color='r', linestyle='--', label='Exact Energy')
plt.title("VQE Cost Function over Optimization Iterations for H₂O")
plt.xlabel("Iteration")
plt.ylabel("Energy (Hartree)")
plt.legend()
plt.grid(True)
plt.show()